In [2]:
!pip install feedparser


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=824ebbea00ba3f536a8e96888fc5fafc4b4c11ffc3d776ddea857f63bada2699
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [3]:
import requests
import feedparser

def search_arxiv(query, max_results=5):
    base_url = "http://export.arxiv.org/api/query?"
    search_query = f"search_query=all:{query.replace(' ', '+')}&start=0&max_results={max_results}"
    query_url = base_url + search_query

    # Parse the response using feedparser (since ArXiv returns Atom XML)
    feed = feedparser.parse(query_url)

    results = []
    for entry in feed.entries:
        paper = {
            "title": entry.title,
            "authors": [author.name for author in entry.authors],
            "summary": entry.summary,
            "published": entry.published,
            "link": entry.link
        }
        results.append(paper)

    return results

#Example
research_question = "large language models for scientific research"
papers = search_arxiv(research_question)

for i, paper in enumerate(papers, 1):
    print(f"\n📄 Paper {i}: {paper['title']}")
    print(f"👨‍🔬 Authors: {', '.join(paper['authors'])}")
    print(f"🗓 Published: {paper['published']}")
    print(f"🔗 Link: {paper['link']}")
    print(f"📝 Abstract:\n{paper['summary'][:500]}...")



📄 Paper 1: A Comprehensive Survey of Scientific Large Language Models and Their
  Applications in Scientific Discovery
👨‍🔬 Authors: Yu Zhang, Xiusi Chen, Bowen Jin, Sheng Wang, Shuiwang Ji, Wei Wang, Jiawei Han
🗓 Published: 2024-06-16T08:03:24Z
🔗 Link: http://arxiv.org/abs/2406.10833v3
📝 Abstract:
In many scientific fields, large language models (LLMs) have revolutionized
the way text and other modalities of data (e.g., molecules and proteins) are
handled, achieving superior performance in various applications and augmenting
the scientific discovery process. Nevertheless, previous surveys on scientific
LLMs often concentrate on one or two fields or a single modality. In this
paper, we aim to provide a more holistic view of the research landscape by
unveiling cross-field and cross-modal co...

📄 Paper 2: The Impact of Large Language Models on Scientific Discovery: a
  Preliminary Study using GPT-4
👨‍🔬 Authors: Microsoft Research AI4Science, Microsoft Azure Quantum
🗓 Published: 2023-11-

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

from transformers import pipeline

# Pipeline
flan_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def extract_with_flan(summary_text):
    prompt = f"""Extract the following details from this summary:
- Task
- Dataset(s)
- Model(s)
- Evaluation metric(s)
- Key result(s)

Summary:
\"\"\"{summary_text}\"\"\"

Respond in bullet points."""

    response = flan_pipe(prompt, max_length=256)[0]['generated_text']
    return response

print("\n🔬 Extracted Experimental Details with FLAN-T5:")
for i, paper in enumerate(papers, 1):
    print(f"\n📄 Paper {i}: {paper['title']}")

    abstract = paper['summary']
    if len(abstract) > 1024:
        abstract = abstract[:1024]

    summary = summarizer(abstract, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    extracted = extract_with_flan(summary)

    print(f"🧪 Experiment Info:\n{extracted}")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu



🔬 Extracted Experimental Details with FLAN-T5:

📄 Paper 1: A Comprehensive Survey of Scientific Large Language Models and Their
  Applications in Scientific Discovery
🧪 Experiment Info:
This paper focuses on the research landscape by examining the interplay between field and modality.

📄 Paper 2: The Impact of Large Language Models on Scientific Discovery: a
  Preliminary Study using GPT-4
🧪 Experiment Info:
This report focuses on GPT-4, the state-of-the-art language model.

📄 Paper 3: Generalization Bias in Large Language Model Summarization of Scientific
  Research
🧪 Experiment Info:
"It's important to understand that large language models are not the only way to support scientific research," said Dr. David A.

📄 Paper 4: SciEval: A Multi-Level Large Language Model Evaluation Benchmark for
  Scientific Research
🧪 Experiment Info:
"SciEval is a comprehensive andmulti-disciplinary evaluation benchmark to address these issues. Based on Bloom's taxonomy, SciEval covers four dimensions t

In [7]:
import pandas as pd

structured_data = []

for i, paper in enumerate(papers, 1):
    entry = {
        "Paper No.": i,
        "Title": paper["title"],
        "Experiment Details": extract_with_flan(
            summarizer(paper["summary"], max_length=150, min_length=40, do_sample=False)[0]["summary_text"]
        )
    }
    structured_data.append(entry)

df = pd.DataFrame(structured_data)

df.head()

,Paper No.,Title,Experiment Details
0,1,A Comprehensive Survey of Scientific Large Lan...,This paper focuses on the research landscape b...
1,2,The Impact of Large Language Models on Scienti...,"This report focuses on GPT-4, the state-of-the..."
2,3,Generalization Bias in Large Language Model Su...,"""LLMs may omitdetails that limit the scope of ..."
3,4,SciEval: A Multi-Level Large Language Model Ev...,"""SciEval is a comprehensive andmulti-disciplin..."
4,5,PaperQA: Retrieval-Augmented Generative Agent ...,Retrieval-Augmented Generation (RAG) models ma...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Paper No.'].plot(kind='hist', bins=20, title='Paper No.')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Experiment Details').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Experiment Details'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='index', y='Experiment Details', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Paper No.', y='Title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Experiment Details'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Paper No.', y='Experiment Details', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [8]:
from transformers import pipeline

recommender = pipeline("text2text-generation", model="google/flan-t5-large")

def recommend_experiment(paper_title, experiment_info):
    prompt = f"""Given the following scientific paper title and experiment summary, suggest a possible next experiment that a researcher could perform.

Title: {paper_title}
Current Experiment: {experiment_info}

Recommended Next Experiment:"""
    result = recommender(prompt, max_length=100, do_sample=False)[0]['generated_text']
    return result

for entry in structured_data:
    entry["Recommended Experiment"] = recommend_experiment(
        entry["Title"],
        entry["Experiment Details"]

df_with_recs = pd.DataFrame(structured_data)
df_with_recs.head()


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


,Paper No.,Title,Experiment Details,Recommended Experiment
0,1,A Comprehensive Survey of Scientific Large Lan...,This paper focuses on the research landscape b...,The paper also discusses the role of the scien...
1,2,The Impact of Large Language Models on Scienti...,"This report focuses on GPT-4, the state-of-the...",The authors suggest that the GPT-4 model is a ...
2,3,Generalization Bias in Large Language Model Su...,"""LLMs may omitdetails that limit the scope of ...",The researcher would like to test whether LLMs...
3,4,SciEval: A Multi-Level Large Language Model Ev...,"""SciEval is a comprehensive andmulti-disciplin...",The authors would like to know if the datasets...
4,5,PaperQA: Retrieval-Augmented Generative Agent ...,Retrieval-Augmented Generation (RAG) models ma...,RAG models can be used to generate scientific ...
